In [1]:
import tensorflow as tf
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pds
from tensorflow import keras
from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperopt import Trials, STATUS_OK, tpe
from tensorflow.keras.optimizers import Adadelta, Nadam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import sys

Using TensorFlow backend.


In [2]:
def create_model(x_train, y_train, x_test, y_test):
    """    
    {'Dense': 2,
 'Dense_1': 0,
 'Dense_2': 2,
 'Dense_3': 1,
 'Dense_4': 1,
 'Dense_5': 0,
 'Dense_6': 2,
 'activation': 2,
 'activation_1': 1,
 'activation_2': 0,
 'activation_3': 2,
 'activation_4': 1,
 'activation_5': 0,
 'activation_6': 2,
 'batch_size': 0,
 'optimizer': 1}
    """
        
    model = Sequential()
    
    model.add(Dense(19, input_shape=(19,)))
    model.add(Dense({{choice([1,19,38])}}, activation={{choice([None, 'relu', 'sigmoid', 'tanh'])}}))
    model.add(Dense({{choice([1,19,38])}}, activation={{choice([None, 'relu', 'sigmoid', 'tanh'])}}))
    model.add(Dense({{choice([1,19,38])}}, activation={{choice([None, 'relu', 'sigmoid', 'tanh'])}}))
    model.add(Dense({{choice([1,19,38])}}, activation={{choice([None, 'relu', 'sigmoid', 'tanh'])}}))
    model.add(Dense({{choice([1,19,38])}}, activation={{choice([None, 'relu', 'sigmoid', 'tanh'])}}))
    model.add(Dense({{choice([1,19,38])}}, activation={{choice([None, 'relu', 'sigmoid', 'tanh'])}}))
    model.add(Dense({{choice([1,19,38])}}, activation={{choice([None, 'relu', 'sigmoid', 'tanh'])}}))
    model.add(Dense(1))

    model.compile(loss='mae', 
                  optimizer={{choice(['RMSProp', 'Adadelta', 'Nadam'])}},
                  metrics=['mae', 'mape']
                 )

    result = model.fit(x_train, y_train,
                   batch_size={{choice([1, 64, 128])}},
                   epochs=7,
                   verbose=1,
                   validation_split=0.1)
    
    loss_metric = np.amax(result.history['mean_absolute_error']) 
    print('Current epoch:', result.history['mean_absolute_error'], result.history['mean_absolute_percentage_error'])
    sys.stdout.flush()
    return {'loss': loss_metric, 'status': STATUS_OK, 'model': model}

In [3]:
def data():
    data = pds.read_pickle('dataset_model2v2_w_label.pkl')
    dataset = data
    labels = dataset['label']
    dataset = dataset.drop(columns=['label'])

    num_test_segments = 3
    msk = np.random.randint(1, dataset.journey_number.unique()[-1], num_test_segments)
    mask = dataset['journey_number'].isin(msk)

    train_input = dataset[~mask]
    test_input = dataset[mask]

    train_labels = labels[~mask]
    test_labels = labels[mask]

    train_data = train_input.drop(columns=['journey_number'])
    test_data = test_input.drop(columns=['journey_number'])

    x_train = train_data
    x_test = test_data
    y_train = train_labels
    y_test = test_labels
    
    return x_train, y_train, x_test, y_test

In [ ]:
best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=30,
                                      trials=Trials(),
                                      notebook_name="Model2v2HPO"
                                     )

>>> Imports:
#coding=utf-8

try:
    import tensorflow as tf
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import pandas as pds
except:
    pass

try:
    from tensorflow import keras
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from tensorflow.keras.optimizers import Adadelta, Nadam
except:
    pass

try:
    from tensorflow.keras import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense
except:
    pass

try:
    import sys
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dense': hp.choice('Dense', [1,19,38]),
        'activation': hp.choice('activation', [None, 'relu', 'sigmoid', 'tanh']),
        'Dense_1': hp.choice('Dense_1', [1,19,38]),
        'activation_1': hp.choice('activation_1', [None, 'relu', 'sigmoid'

Epoch 6/7
2978307/2978307 [==============================] - 33s 11us/step - loss: 11.4622 - mean_absolute_error: 11.4622 - mean_absolute_percentage_error: 21622463.5521 - val_loss: 12.3172 - val_mean_absolute_error: 12.3172 - val_mean_absolute_percentage_error: 31017728.0415
Epoch 7/7
2978307/2978307 [==============================] - 33s 11us/step - loss: 11.3994 - mean_absolute_error: 11.3994 - mean_absolute_percentage_error: 20380080.2492 - val_loss: 12.0981 - val_mean_absolute_error: 12.0981 - val_mean_absolute_percentage_error: 38382954.7719
Current epoch: [15.155696479477239, 12.07207254423598, 11.793298436436006, 11.63430842820569, 11.541610121747423, 11.462218480733311, 11.399442122842744] [122321401.7509339, 39679713.11936867, 29965444.592538845, 24724060.212910257, 22964270.972591687, 21622463.552101765, 20380080.2491604]
Train on 2978307 samples, validate on 330923 samples
Epoch 1/7
2978307/2978307 [==============================] - 41s 14us/step - loss: 37.4179 - mean_abso

2978307/2978307 [==============================] - 42s 14us/step - loss: 37.0003 - mean_absolute_error: 37.0003 - mean_absolute_percentage_error: 516728154.5247 - val_loss: 36.6987 - val_mean_absolute_error: 36.6987 - val_mean_absolute_percentage_error: 518328637.9823
Epoch 3/7
2978307/2978307 [==============================] - 41s 14us/step - loss: 37.0002 - mean_absolute_error: 37.0002 - mean_absolute_percentage_error: 516992481.7977 - val_loss: 36.7052 - val_mean_absolute_error: 36.7052 - val_mean_absolute_percentage_error: 526870052.7033
Epoch 4/7
2978307/2978307 [==============================] - 41s 14us/step - loss: 37.0005 - mean_absolute_error: 37.0005 - mean_absolute_percentage_error: 516896191.1760 - val_loss: 36.7056 - val_mean_absolute_error: 36.7056 - val_mean_absolute_percentage_error: 527304889.7125
Epoch 5/7
2978307/2978307 [==============================] - 42s 14us/step - loss: 37.0000 - mean_absolute_error: 37.0000 - mean_absolute_percentage_error: 517193406.9632 - 

Epoch 7/7
2978307/2978307 [==============================] - 33s 11us/step - loss: 36.9988 - mean_absolute_error: 36.9988 - mean_absolute_percentage_error: 515531862.5862 - val_loss: 36.6989 - val_mean_absolute_error: 36.6989 - val_mean_absolute_percentage_error: 518027233.4728
Current epoch: [37.637532660790036, 36.99880795863666, 36.99879699984717, 36.99878746731793, 36.99881464954909, 36.99880354924482, 36.99878781371515] [490294391.78598654, 515497147.465233, 515511050.75757796, 515786712.6700247, 515718287.8672892, 515657476.0937397, 515531862.5862048]
Train on 2978307 samples, validate on 330923 samples
Epoch 1/7
2978307/2978307 [==============================] - 4082s 1ms/step - loss: 37.3709 - mean_absolute_error: 37.3709 - mean_absolute_percentage_error: 520496655.6353 - val_loss: 36.7641 - val_mean_absolute_error: 36.7641 - val_mean_absolute_percentage_error: 549119029.3131
Epoch 2/7
2978307/2978307 [==============================] - 4116s 1ms/step - loss: 37.2814 - mean_abso

In [ ]:
best_run